### Simple example for performing symbolic regression for a set of points

In [11]:
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation
from pathlib import Path
from functools import partial
import torch
from sympy import lambdify

In [12]:
## Use same set of configuration parameters of ours test set
test_data = load_metadata_hdf5("../data/validation")
params_fit = FitParams(word2id=test_data.word2id, 
                            id2word=test_data.id2word, 
                            una_ops=test_data.una_ops, 
                            bin_ops=test_data.bin_ops, 
                            total_variables=list(test_data.total_variables),  
                            total_coefficients=list(test_data.total_coefficients),
                            rewrite_functions=list(test_data.rewrite_functions)
                            )

In [13]:
# Load (Hydra) yaml file with all architecture parameters details
import omegaconf
cfg = omegaconf.OmegaConf.load("../scripts/config.yaml")

In [14]:
weights_path = "../weights/100M.ckpt"

In [15]:
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()

In [16]:
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [17]:
# Create points from an equation
number_of_points = 500
n_variables = 2

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(test_data.total_variables)))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "x_1*sin(x_2/2)" #Use x_1,x_2 and x_3 as indepdent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(test_data.total_variables)} #CHECK ME
y = lambdify(",".join(test_data.total_variables), target_eq)(**X_dict)

In [18]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  torch.Size([500, 3])
y shape:  torch.Size([500])


In [21]:
output = fitfunc(X,y) 

Memory footprint of the encoder: 4.096e-05GB 

Constructing BFGS loss...
Flag idx remove ON, Removing indeces with high values...
checking input values range...
Loss constructed, starting new BFGS optmization...


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
output